In [1]:
import xgboost as xgb
import numpy as np
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
import gym
from gym import spaces
import torch
import numpy as np
from scipy import stats


from stable_baselines3.common.vec_env import DummyVecEnv






In [2]:
n_samples = 5000
n_features = 53
n_informative = 53
n_targets = 27
noise = 0.1
random_seed= np.random.randint(0,1000)


X, y = make_regression(n_samples=n_samples,n_features=n_features,n_informative=n_informative,n_targets=n_targets, noise=noise,random_state=random_seed)

reg = xgb.XGBRegressor(n_estimators=100,multi_strategy="multi_output_tree")
#reg = xgb.XGBRFRegressor(n_estimators=100,multi_strategy="multi_output_tree")
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=random_seed)
# X_val, _, y_val, _ = train_test_split(
#     X_train, y_train, test_size=0.33, random_state=random_seed)

In [3]:
num_instance = X_train.shape[0]
random_index = np.random.randint(0,num_instance, size=int(num_instance*0.7))
X_train[random_index,:]#.shape

array([[-0.14003994,  1.76680471, -1.06805467, ...,  0.15280249,
         1.83418775,  0.69882047],
       [ 0.17882637, -2.90555598,  0.21679232, ...,  1.70063301,
        -1.00764567,  1.07931188],
       [ 0.23105181,  1.15996979, -1.44475941, ..., -1.57306153,
         1.30265932, -0.88786769],
       ...,
       [-1.09910078, -0.78761912,  0.76475921, ..., -0.9018305 ,
         0.14599873,  0.12654548],
       [-1.36126052, -0.2498196 , -0.9105065 , ...,  1.18645821,
         0.15490431, -2.16598927],
       [-0.23577227,  2.46866423,  0.94201001, ...,  0.46260751,
        -0.74016035, -0.95571328]])

In [45]:
from sklearn.preprocessing import StandardScaler


StandardScaler()

StandardScaler()

In [63]:
n_samples = 5000
n_features = 53
n_informative = 53
n_targets = 27
noise = 0.1
random_seed= np.random.randint(0,1000)


X, y = make_regression(n_samples=n_samples,n_features=n_features,n_informative=n_informative,n_targets=n_targets, noise=noise,random_state=random_seed)
scaler = StandardScaler()
scaler.fit(X)
X_t = scaler.transform(X)
scaler.fit(y)
y_t = scaler.transform(y)
reg = xgb.XGBRegressor(n_estimators=100,multi_strategy="multi_output_tree")
#reg = xgb.XGBRFRegressor(n_estimators=100,multi_strategy="multi_output_tree")
X_train, X_test, y_train, y_test = train_test_split(
    X_t, y_t, test_size=0.33, random_state=random_seed)
reg.fit(X_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy='multi_output_tree', n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [64]:
y_pred = reg.predict(X_test)
from sklearn.metrics import r2_score
r2_score(y_test, y_pred,multioutput='raw_values')

array([0.63929998, 0.63945625, 0.63966271, 0.60220381, 0.62236028,
       0.6398983 , 0.63636454, 0.604517  , 0.61265981, 0.60018841,
       0.62898565, 0.59886079, 0.60794053, 0.62101997, 0.62604487,
       0.62409749, 0.63953335, 0.62848115, 0.64465526, 0.60734398,
       0.62709051, 0.63080514, 0.6303681 , 0.61970487, 0.63698301,
       0.62667627, 0.61579597])

In [78]:
class CustomEnv(gym.Env):
    def __init__(self, S, A, model, max_step=100):
        #self.S = torch.tensor(S, dtype=torch.float32)  # Convert S to torch tensor
        #self.A = torch.tensor(A, dtype=torch.float32)  # Convert A to torch tensor
        
        self.S = self._normalize(torch.tensor(S, dtype=torch.float32, device="cpu"))#.to(self.device)
        self.A = self._normalize(torch.tensor(A, dtype=torch.float32))#.to(self.device)
       
        self.num_samples = int(self.S.shape[0]*0.7)### 0.7 之後當參數
        self.current_step = 0
        self.model = model
        self.max_step = max_step
        self.action_instance = int(self.A.shape[0]*0.7)
        self.action_dim = self.A.shape[1]
        self.action_space = spaces.Box(low=-1e6, high=1e6, shape=(self.num_samples,self.action_dim))
        
        self.obs_dim_S = self.S.shape[1]
     
        self.observation_space = spaces.Box(low=-1e6, high=1e6, shape=(self.num_samples,self.obs_dim_S))
        #print("DONE 1")
        
    # def reset(self):
    #     self.current_step = 0
    #     print("DONE 2")
    #     return self.S.numpy() #self._get_observation()

    def _get_observation(self):
        # self.current_step 放哪?
        num_instance = self.S.shape[0]
        random_index = np.random.randint(0,num_instance, size=int(self.num_samples))
        obs = self.S[random_index,:]#.shape 
        #print("DONE 2")    
        return obs,random_index
    def reset(self):
        self.current_step = 0
        obs , _ = self._get_observation()
        #print("DONE 3")  
        return obs
    def step(self, action):
        next_state, _ = self._get_observation() # _ 為random_index，可能對action有差
        action = torch.tensor(action, dtype=torch.float32)
        denorm_action = self._denormalize(action, self.A)
        #print("next_state step",next_state.shape)
        #print("action step",action.shape)
        reward,score = self._calculate_reward(action)
        
        best_action = None
        best_score = -np.inf
        if (reward ==1) and (score > best_score):
            best_score = score
            best_action = denorm_action #action
            

        done = (self.current_step == self.max_step - 1)
        self.current_step = self.current_step + 1
        #print("DONE 4")  
        return next_state, reward, done, {"best_action":best_action,"best_score":best_score}
    def _calculate_reward(self, action):


        next_state, obs_index = self._get_observation()
        #print("next_state",next_state.shape)
        #print("action ",action.numpy().shape)
        #print("A",self.A.numpy()[obs_index,:].shape)

        #print(action.numpy())
        y_pred_observe = self.model.predict(action.numpy())# action.numpy()
        y_pred = self.model.predict(self.A.numpy()[obs_index,:])

        denorm_S = self._denormalize(next_state, next_state)
        denorm_y_pred_observe = self._denormalize(y_pred_observe, next_state)
        denorm_y_pred = self._denormalize(y_pred, next_state)
        #print(np.isnan(y_pred_observe).any())
        #print(np.isnan(y_pred).any())
        #print(next_state)
        r2_observe = r2_score(denorm_S.numpy(), denorm_y_pred_observe, multioutput='raw_values')
        r2_ground = r2_score(denorm_S.numpy(), denorm_y_pred,multioutput='raw_values')
        #print(r2_observe,r2_ground)
        score = None
        if stats.ttest_ind(r2_observe, r2_ground, alternative="greater").pvalue < 0.05:
            reward = 1
            score = r2_observe
        elif stats.ttest_ind(r2_observe, r2_ground, alternative="less").pvalue < 0.05:
            reward = -1
        else:
            reward = 0
        
        return reward, score

    def _normalize(self, tensor):
        mean = tensor.mean(dim=0, keepdim=True)
        std = tensor.std(dim=0, keepdim=True)
        #print(tensor)
        return (tensor - mean) / (std + 1e-8)  # Add small epsilon to avoid division by zero
    
    def _denormalize(self, tensor, original):
        mean = original.mean(dim=0, keepdim=True)
        std = original.std(dim=0, keepdim=True)
        print(tensor * (std + 1e-8) + mean)
        return tensor * (std + 1e-8) + mean
    # def step(self, action):
    #     # Apply action and get next state and reward
    #     next_state = self._get_observation()
    #     action = torch.from_numpy(action.astype(np.float64) )
    #     reward = self._calculate_reward(action)
    #     #print("next_state",next_state.shape)
    #     #print("action",action.shape)
    #     #print("reward",reward.shape)
    #     #print("self.current_step",self.current_step)
    #     done = (self.current_step == self.obs_dim_S - 1)
    #     #print("done",done)
    #     self.current_step = self.current_step + 1
    #     #print("self.current_step",self.current_step)
    #     return next_state, reward, done, {}



In [52]:
stats.ttest_ind(np.random.randint(10,20,10), np.random.randint(2,10,10),alternative="less").pvalue

0.9999999648197204

In [68]:
import torch
a = torch.ones((1,3))
print(a)
a

tensor([[1., 1., 1.]])


tensor([[1., 1., 1.]])

In [66]:
from sb3_contrib import ARS # 機率輸出不會改變
from stable_baselines3 import A2C # 機率輸出會改變
from stable_baselines3 import DDPG # 輸出機率和動作長依樣?
from stable_baselines3 import HER # 這要搭配其他使用
from stable_baselines3 import PPO # 機率輸出會改變
from stable_baselines3 import SAC # 機率輸出會改變
from stable_baselines3 import TD3 # 輸出機率和動作長依樣?
from sb3_contrib import TRPO
from sb3_contrib import TQC
from sb3_contrib import RecurrentPPO # MlpLstmPolicy


In [79]:
RL_model = PPO

env = CustomEnv(S=y_train, A=X_train, model=reg, max_step=100)

# Wrap the environment in a vectorized form
venv = DummyVecEnv([lambda: env])

print(RL_model)
# Initialize and train the RL agent (using PPO as an example)
model = RL_model("MlpPolicy", venv, verbose=1) # MultiInputPolicy
model.learn(total_timesteps=10000)

<class 'stable_baselines3.ppo.ppo.PPO'>
Using cpu device
tensor([[ 0.0804, -1.0730, -1.1698,  ..., -0.8861, -0.7540, -1.4322],
        [-0.0324,  0.6113, -1.0545,  ...,  0.5644,  0.7266, -0.6310],
        [ 1.4305,  0.4664,  1.4826,  ...,  1.5193,  0.6011,  0.8575],
        ...,
        [ 0.5943, -0.7795,  1.4454,  ...,  0.2164,  1.6991, -0.6598],
        [ 0.1754, -0.1361, -1.0897,  ..., -1.1861,  0.5129, -0.5533],
        [ 0.3713, -2.8223,  1.0299,  ...,  0.0512, -2.3023, -0.6202]])
tensor([[-0.4056, -1.6624, -0.9753,  ..., -0.0026,  0.2531, -1.0606],
        [-0.6332,  0.1490, -1.1486,  ..., -0.0703, -0.4821, -1.0673],
        [-1.3215, -1.4303, -2.0153,  ..., -0.1992, -1.8041, -0.7613],
        ...,
        [-1.6780,  0.0533, -1.1117,  ...,  0.2507, -0.4688, -0.3743],
        [-1.4107, -0.9495, -1.6938,  ...,  0.0230, -1.4372, -1.6598],
        [-0.6561, -0.6578, -1.6417,  ..., -0.4803, -1.2477,  0.0235]])


TypeError: unsupported operand type(s) for *: 'numpy.ndarray' and 'Tensor'

In [220]:
import gym
from gym import spaces
import numpy as np
import torch
from sklearn.metrics import r2_score
from scipy import stats

class CustomEnv(gym.Env):
    def __init__(self, S, A, model, max_step=100, device='cpu'):
        self.device = device
        
        # Normalize S and A
        self.S = torch.tensor(S, dtype=torch.float32)#self._normalize(torch.tensor(S, dtype=torch.float32)).to(self.device)
        self.A = torch.tensor(A, dtype=torch.float32)#self._normalize(torch.tensor(A, dtype=torch.float32)).to(self.device)
        
        self.num_samples = self.S.shape[0]
        self.current_step = 0
        self.model = model
        self.max_step = max_step
        self.action_instance = self.A.shape[0]
        self.action_dim = self.A.shape[1]
        
        # 计算边界（假设数据大致遵循正态分布）
        self.obs_low = float(self.S.min())*1.5
        self.obs_high = float(self.S.max())*1.5
        self.action_low = float(self.A.min())*1.5
        self.action_high = float(self.A.max())*1.5

        # Limit the action space
        self.action_space = spaces.Box(low=self.action_low, high=self.action_high, shape=(self.action_instance, self.action_dim), dtype=np.float32)
        
        self.obs_dim_S = self.S.shape[1]
        self.observation_space = spaces.Box(low=self.obs_low, high=self.obs_high, shape=(self.num_samples, self.obs_dim_S), dtype=np.float32)
        
    # def _normalize(self, tensor):
    #     mean = tensor.mean(dim=0, keepdim=True)
    #     std = tensor.std(dim=0, keepdim=True)
    #     return (tensor - mean) / (std + 1e-8)  # Add small epsilon to avoid division by zero
    
    # def _denormalize(self, tensor, original):
    #     mean = original.mean(dim=0, keepdim=True)
    #     std = original.std(dim=0, keepdim=True)
    #     return tensor * (std + 1e-8) + mean

    def reset(self):
        self.current_step = 0
        return self.S.cpu().numpy()
    
    def step(self, action):
        # 确保动作在边界内
        action = np.clip(action, -1, 1)
        action = torch.tensor(action, dtype=torch.float32).to(self.device)
        
        # 在使用之前反标准化动作
        denorm_action = action#self._denormalize(action, self.A)
        
        reward, score = self._calculate_reward(denorm_action)
        print("STEP.....","reward: ",reward,"score: ",score)
        best_action = None
        best_score = float('-inf')
        if reward > 0 and score is not None:
            # 如果 score 是张量，取其平均值
            current_score = score.mean().item() if isinstance(score, torch.Tensor) else score
            if current_score > best_score:
                best_score = current_score
                best_action = denorm_action.cpu().numpy()  # 将最佳动作转换为 numpy 数组

        done = (self.current_step == self.max_step - 1)
        self.current_step += 1
        
        return self.S.cpu().numpy(), reward, done, {"best_action": best_action, "best_score": best_score}
    
    def _calculate_reward(self, action):
        try:
            y_pred_observe = torch.tensor(self.model.predict(action.cpu().numpy()), dtype=torch.float32).to(self.device)
            y_pred = torch.tensor(self.model.predict(self.A.cpu().numpy()), dtype=torch.float32).to(self.device)

            # 反标准化预测和真实值以进行正确的 R2 计算
            # denorm_S = self._denormalize(self.S, self.S)
            # denorm_y_pred_observe = self._denormalize(y_pred_observe, self.S)
            # denorm_y_pred = self._denormalize(y_pred, self.S)

            r2_observe = r2_score(self.S.cpu().numpy(), y_pred_observe.cpu().numpy(), multioutput='raw_values')
            r2_ground = r2_score(self.S.cpu().numpy(), y_pred.cpu().numpy(), multioutput='raw_values')

            # 检查 NaN 值
            if np.isnan(r2_observe).any() or np.isnan(r2_ground).any():
                return 0.0, None

            score = None
            print("r2_observe: ",r2_observe,"r2_ground: ",r2_ground)
            if stats.ttest_ind(r2_observe, r2_ground, alternative="greater").pvalue < 0.05:
                reward = float(np.mean(r2_observe) - np.mean(r2_ground))  # 连续奖励
                score = r2_observe
                
            elif stats.ttest_ind(r2_observe, r2_ground, alternative="less").pvalue < 0.05:
                reward = float(np.mean(r2_observe) - np.mean(r2_ground))  # 负奖励
            else:
                reward = 0.0
            
            return reward, torch.tensor(score).to(self.device) if score is not None else None
        
        except Exception as e:
            print(f"Error in _calculate_reward: {e}")
            return 0.0, None

In [106]:
torch.tensor([[1,2,3],[1,2,3]]).to("cpu").numpy()

array([[1, 2, 3],
       [1, 2, 3]], dtype=int64)

In [ ]:
# Generate synthetic data
n_samples = 5000
n_features = 53
n_informative = 53
n_targets = 27
noise = 0.1
random_seed = np.random.randint(0, 1000)

X, y = make_regression(n_samples=n_samples, n_features=n_features, n_informative=n_informative, n_targets=n_targets, noise=noise, random_state=random_seed)

reg = xgb.XGBRegressor(n_estimators=100)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=random_seed)
reg.fit(X_train, y_train)


In [218]:
reg = xgb.XGBRegressor(n_estimators=100,multi_strategy="multi_output_tree")
#reg = xgb.XGBRFRegressor(n_estimators=100,multi_strategy="multi_output_tree")
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.33, random_state=random_seed)
reg.fit(X,y)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy='multi_output_tree', n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [221]:

env = CustomEnv(S=y, A=X, model=reg, max_step=100, device='cpu')
venv = DummyVecEnv([lambda: env])

# 调整 PPO 参数
model = A2C("MlpPolicy", venv, verbose=1, 
            learning_rate=1e-4,  # 降低学习率
            #n_steps=2048,  # 增加每次更新的步数
            #batch_size=64,  # 调整批量大小
            #n_epochs=50,  # 每次更新的 epoch 数
            #gamma=0.99,  # 折扣因子
            #gae_lambda=0.95,  # GAE lambda 参数
            #clip_range=0.2,  # PPO clip 参数
            #ent_coef=0.01,  # 熵系数
            #vf_coef=0.5,  # 价值函数系数
            #max_grad_norm=0.5  # 梯度裁剪
           )

# 增加总时间步
model.learn(total_timesteps=50000)

c:\Users\foresight_User\anaconda3\envs\Python3.9.13\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Using cpu device


In [186]:
best_rewards = float('-inf')
best_action = None
best_score = float('-inf')

obs = np.array([y_test])#venv.reset()
while True:
    action, _states = model.predict(obs, deterministic=True)
    obs, rewards, dones, infos = venv.step(action)
    print(action)
    for i, done in enumerate(dones):
        if not done:
            if infos[0]['best_score'] > best_score:
                best_score = infos[0]['best_score']
                best_action = infos[0]['best_action']
                best_rewards = rewards
        else:
            print(f"Episode finished. Best reward: {best_rewards}")
            obs = np.array([y_test])#venv.reset()
    
    if all(dones):
        break

print(f"Final best reward: {best_rewards}")
print(f"Best action shape: {best_action.shape}")

[[[-6.4416445e-04 -4.4253690e-04  5.3753465e-04 ... -5.8452744e-04
    8.7908289e-04 -7.2297093e-04]
  [ 2.2914370e-04  1.1339259e-04 -4.5065931e-04 ...  7.8272994e-04
   -1.8573302e-04  2.1589229e-04]
  [-5.9246237e-04  3.6587234e-04  1.1118212e-04 ...  1.3101724e-04
   -2.5329209e-04 -4.0883373e-04]
  ...
  [ 1.6246187e-04 -4.6094184e-04 -3.0144365e-04 ... -3.7656029e-04
   -1.0011983e-04  5.2849302e-04]
  [ 7.6623837e-06 -8.0701255e-04 -8.6149375e-05 ...  1.8230599e-04
   -1.7657783e-04 -2.6060143e-04]
  [ 1.2534023e-03  4.1754203e-04 -5.6128646e-04 ... -1.7320376e-04
   -1.3502008e-04 -2.4259021e-04]]]
[[[-5.1394961e-04 -4.7608704e-04  5.2995764e-04 ... -6.0575421e-04
    9.2502235e-04 -6.8890757e-04]
  [ 3.3420537e-04  3.8812559e-05 -3.4941011e-04 ...  8.6757226e-04
   -1.1880247e-04  2.1778324e-04]
  [-5.4145965e-04  4.1917906e-04 -3.5370711e-05 ...  1.0061478e-04
   -2.3408164e-04 -2.7884293e-04]
  ...
  [ 2.7059173e-04 -5.1295175e-04 -1.5497486e-04 ... -4.2141596e-04
   -6.4845

In [202]:
X_ = X_test
y_ = y_test
X_train_, X_test_, y_train_, y_test_ = train_test_split(
    X_,y_, test_size=0.33, random_state=random_seed)

reg = xgb.XGBRegressor(n_estimators=100,multi_strategy="multi_output_tree")
reg.fit(X_train_,y_train_)

r2_score(y_test_, reg.predict(X_test_),multioutput='raw_values') # 

array([0.44114839, 0.44216944, 0.45684991, 0.47484812, 0.45862346,
       0.39800638, 0.46185   , 0.45723812, 0.42179091, 0.46475374,
       0.42595316, 0.40751603, 0.40290237, 0.43868479, 0.41647986,
       0.41246714, 0.43777254, 0.41986104, 0.42451871, 0.44092706,
       0.41921476, 0.41173098, 0.43145455, 0.44548968, 0.43048148,
       0.45100258, 0.41429267])

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy='multi_output_tree', n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

0.9966452080459303

In [203]:
X_ = best_action
y_ = y_test
X_train_, X_test_, y_train_, y_test_ = train_test_split(
    X_,y_, test_size=0.33, random_state=random_seed)

reg = xgb.XGBRegressor(n_estimators=100,multi_strategy="multi_output_tree")
reg.fit(X_train_,y_train_)

r2_score(y_test_, reg.predict(X_test_),multioutput='raw_values') # 

array([-0.14977162, -0.20545601, -0.16800556, -0.16584538, -0.22857052,
       -0.18538891, -0.19423067, -0.18974981, -0.20921431, -0.19452683,
       -0.20434574, -0.15557048, -0.1906581 , -0.17777691, -0.17420653,
       -0.21368858, -0.18323978, -0.17466585, -0.16301171, -0.24831262,
       -0.18820203, -0.21340787, -0.21311911, -0.20724806, -0.19595961,
       -0.19675777, -0.20039371])

0.9966452080459303

In [200]:
r2_score(y_test, reg2.predict(best_action))

0.9364943039932776

In [175]:


#y_pred = reg.predict(X_test)
#from sklearn.metrics import r2_score
r2_score(obs[0,:,:], reg.predict(best_action),multioutput='raw_values')

array([-2.4891643e+04, -3.5539529e+02, -6.0431548e+03, -2.6582246e+04,
       -5.8130642e+01, -2.7075613e+04, -7.1159552e+02, -7.9571768e+03,
       -3.2896067e+03, -2.7218879e+04, -2.3255133e+01, -5.4125085e+02,
       -3.6726370e+03, -1.4152050e+04, -2.1654193e+04, -8.3820502e+02,
       -7.5444995e+03, -1.3972151e+04, -7.1884900e+02, -1.7498598e+04,
       -7.9152626e+01, -9.2042404e+01, -3.4865093e+03, -2.8647232e+02,
       -3.8316996e+04, -1.8346722e+01, -2.3762852e+01], dtype=float32)

In [180]:
action, _states = model.predict(obs, deterministic=True)

In [182]:
action.shape

(1, 1650, 53)

In [185]:
np.array([y_test]).shape

(1, 1650, 27)

In [176]:
reg.predict(best_action)

array([[ 157.723    ,  -18.846218 ,   77.71419  , ..., -195.67908  ,
          -4.2820086,   -4.873236 ],
       [ 157.723    ,  -18.846218 ,   77.71419  , ..., -195.67908  ,
          -4.2820086,   -4.873236 ],
       [ 157.723    ,  -18.846218 ,   77.71419  , ..., -195.67908  ,
          -4.2820086,   -4.873236 ],
       ...,
       [ 157.723    ,  -18.846218 ,   77.71419  , ..., -195.67908  ,
          -4.2820086,   -4.873236 ],
       [ 157.723    ,  -18.846218 ,   77.71419  , ..., -195.67908  ,
          -4.2820086,   -4.873236 ],
       [ 157.723    ,  -18.846218 ,   77.71419  , ..., -195.67908  ,
          -4.2820086,   -4.873236 ]], dtype=float32)

In [171]:

#reg = xgb.XGBRFRegressor(n_estimators=100,multi_strategy="multi_output_tree")
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=random_seed)
reg.fit(X_train,y_train)

array([[[-1.9440656 , -2.4832075 , -2.7457654 , ..., -2.4329946 ,
         -1.8794603 , -1.9379606 ],
        [ 0.36553282, -0.4712124 , -0.38807377, ...,  0.34913057,
         -0.58862156, -0.3183774 ],
        [ 1.1339395 ,  0.89081615,  1.8997779 , ...,  1.4395908 ,
          1.3171606 ,  1.1785853 ],
        ...,
        [-0.16502228,  0.1973744 , -0.22131212, ...,  0.6220481 ,
          1.4465963 ,  0.93446815],
        [-1.2392161 , -0.96861684, -1.2825663 , ..., -1.5186218 ,
         -1.662533  , -1.644962  ],
        [ 0.67251164, -0.02450459,  1.4685475 , ...,  0.37250558,
          0.8097894 ,  0.7944904 ]]], dtype=float32)

In [170]:
y_test

array([[ -745.09270589, -1023.5077659 , -1020.07824892, ...,
        -1172.38410318,  -738.02856596,  -783.21412941],
       [  150.70465513,  -184.7174612 ,  -144.47184494, ...,
          169.63265957,  -220.95496124,  -122.82454254],
       [  448.73765614,   383.10520629,   705.19714621, ...,
          695.63915894,   542.44758089,   487.56618973],
       ...,
       [  -55.07566166,    94.01292003,   -82.53941551, ...,
          301.28018067,   594.29584932,   388.02675469],
       [ -471.71084416,  -392.08282656,  -476.67106989, ...,
         -731.31708395,  -651.1336304 ,  -663.74311863],
       [  269.76898453,     1.51271344,   545.04555344, ...,
          180.90809987,   339.2090263 ,   330.95045377]])

In [169]:
y_test

array([[ -745.09270589, -1023.5077659 , -1020.07824892, ...,
        -1172.38410318,  -738.02856596,  -783.21412941],
       [  150.70465513,  -184.7174612 ,  -144.47184494, ...,
          169.63265957,  -220.95496124,  -122.82454254],
       [  448.73765614,   383.10520629,   705.19714621, ...,
          695.63915894,   542.44758089,   487.56618973],
       ...,
       [  -55.07566166,    94.01292003,   -82.53941551, ...,
          301.28018067,   594.29584932,   388.02675469],
       [ -471.71084416,  -392.08282656,  -476.67106989, ...,
         -731.31708395,  -651.1336304 ,  -663.74311863],
       [  269.76898453,     1.51271344,   545.04555344, ...,
          180.90809987,   339.2090263 ,   330.95045377]])